In [ ]:
# these are the nodes that are either the start of the line or the end of the line
    # now, we compute some distances
    se_nodes = n_edge_count_df['node_id'].tolist()
    node_dist_list = []
    for sn_id, en_id in combinations(se_nodes, 2):                        
        node_dist = points2distance(node_dict[sn_id], node_dict[en_id], unit = 'miles')
        node_dist_list.append([sn_id, en_id, node_dist])

    node_dist_df = pd.DataFrame(data = node_dist_list, 
                                columns = ['sn_id', 'en_id', 'dist_miles'])
    
    # so, this edge list is the anti-graph. 
    sn_id_agg_df = node_dist_df[['sn_id', 'dist_miles']].groupby(['sn_id']).agg(tot_dist = ('dist_miles', 'sum')).reset_index()
    en_id_agg_df = node_dist_df[['en_id', 'dist_miles']].groupby(['en_id']).agg(tot_dist = ('dist_miles', 'sum')).reset_index()
    # it contains the missing links and then some. What I need to do is find the shortest path that connects everything.
    # let's first start with the node that is the farthest away from everything.

    max_dist_list = [sn_id_agg_df['tot_dist'].max(), en_id_agg_df['tot_dist'].max()]
    max_dist_id = np.argmax(max_dist_list)
    max_dist = np.max(max_dist_list)
    if max_dist_id == 0:
        fp_df = sn_id_agg_df.loc[sn_id_agg_df['tot_dist'] == max_dist, :]
        fp = fp_df['sn_id'].iloc[0]    
    else:
        fp_df = en_id_agg_df.loc[en_id_agg_df['tot_dist'] == max_dist, :]
        fp = fp_df['en_id'].iloc[0]
    
    print(fp)
    node_dist_df['sn_snd_group_id'] = node_dist_df['sn_id'].map(node_snd_group_dict)
    node_dist_df['en_snd_group_id'] = node_dist_df['en_id'].map(node_snd_group_dict)
    
    # remove distances between nodes on the same edge
    node_dist_df = node_dist_df.loc[node_dist_df['sn_snd_group_id'] != node_dist_df['en_snd_group_id'], :]
    print(node_dist_df)
    
    current_node = fp
    current_edge = node_snd_group_dict[fp]
    #print(current_node, current_edge)    
        
    # start visiting!
    # the farthest point
    visited_nodes = []
    draw_nodes = []
    # edge of the farthest point
    visited_edges = []
                     
    nodes_to_visit = list(node_snd_group_dict.keys())
    edges_to_visit = list(n_edge_count_df['snd_group'].unique().tolist())
    escape_value = len(visited_nodes)
    loop_count = 0
    
    while (len(visited_edges) < (len(edges_to_visit) - 1)):
        loop_count += 1
        print(loop_count, current_node, current_edge)    
        # remove the current node from the choices
        node_dist_df = node_dist_df.loc[(node_dist_df['sn_id'] != current_node) &
        (node_dist_df['en_id'] != current_node), :]    
        
        # the list of nodes on the current edge
        current_node_list = snd_group_node_dict[current_edge]
        
        # the remaining node is the next node
        if current_node_list[0] == current_node:
            next_node = current_node_list[1]
        else:
            next_node = current_node_list[0]
    
        visited_nodes.append(current_node)
        visited_nodes.append(next_node)
        visited_edges.append(current_edge)        
        
        # find the closest node on a different edge    
        temp_node_dist = node_dist_df.loc[(node_dist_df['sn_id'] == next_node) |
        (node_dist_df['en_id'] == next_node), :].copy()
        #print(temp_node_dist.head())
    
        # drop the next node
        node_dist_df = node_dist_df.loc[(-node_dist_df['sn_id'].isin(visited_nodes)) &
        (-node_dist_df['en_id'].isin(visited_nodes)), :].copy()     
    
        temp_node_dist = temp_node_dist.loc[temp_node_dist['dist_miles'] == temp_node_dist['dist_miles'].min(), :]
    
        if temp_node_dist.shape[0] > 0:
    
            if temp_node_dist['sn_id'].iloc[0] == next_node:
                current_node = temp_node_dist['en_id'].iloc[0]
                current_edge = temp_node_dist['en_snd_group_id'].iloc[0]        
            else:
                current_node = temp_node_dist['sn_id'].iloc[0]
                current_edge = temp_node_dist['sn_snd_group_id'].iloc[0]
            
        #print(visited_nodes)
        #print(visited_edges)       
        
    # missing segments!
    # but we need to go: 1-2, 3-4, 
    id_list = []
    data_list = []
    line_list = []
    for i_cn, cn in enumerate(visited_nodes[:-1]):
        #print(cn)
        id_list.append(i_cn)
        nn = visited_nodes[i_cn + 1]
        temp_data_list = [sn, i_cn, cn, nn, node_snd_group_dict[cn], node_snd_group_dict[nn]]
        my_line_string = LineString([node_dict[cn], node_dict[nn]])
        line_list.append(my_line_string)
        data_list.append(temp_data_list)   